# Solana Security Incidents Analysis Notebook

This notebook provides tools and analysis for investigating security incidents on the Solana blockchain, including hacks, exploits, and vulnerabilities.

## Overview

Security incidents on Solana typically fall into several categories:

1. **Smart Contract Exploits**: Vulnerabilities in program code that allow attackers to manipulate functionality
2. **Economic Attacks**: Flash loans, price manipulation, and oracle attacks
3. **Private Key Compromises**: Theft or compromise of private keys
4. **Social Engineering**: Phishing attacks targeting Solana users
5. **Protocol Vulnerabilities**: Issues in underlying infrastructure or protocols

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import networkx as nx
import requests
from typing import List, Dict, Any, Optional, Tuple

# Add project root to path
sys.path.append('../..')

# Import SolanaGuard modules
from data_collection.collectors.helius_collector import HeliusCollector
from data_collection.collectors.rugcheck_collector import RugCheckCollector
from data_collection.collectors.vybe_collector import VybeCollector
from utils.graph_utils import TransactionFlowGraph
from utils.risk_scoring import calculate_address_risk
from utils.visualization import visualize_transaction_flow
from utils.entropy_analysis import calculate_transaction_entropy, detect_entropy_anomalies
from utils.incident_analysis import analyze_security_incident, identify_vulnerability_patterns, plot_incident_timeline

# Configure plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

ModuleNotFoundError: No module named 'collectors'

## Initialize API Collectors

First, let's initialize the necessary API collectors to gather data for our analysis:

In [ ]:
# Initialize collectors
from data_collection.collectors.range_collector import RangeCollector
from data_collection.collectors.helius_collector import HeliusCollector
from data_collection.collectors.rugcheck_collector import RugCheckCollector
from data_collection.collectors.vybe_collector import VybeCollector

helius = HeliusCollector()
range_api = RangeCollector()
vybe = VybeCollector()

print("Collectors initialized successfully")

## Security Incident Database

Let's create a database of known security incidents on Solana for analysis:

In [ ]:
# Database of major Solana security incidents
SECURITY_INCIDENTS = [
    {
        "name": "Wormhole Bridge Exploit",
        "date": "2022-02-02",
        "type": "bridge_exploit",
        "loss_usd": 320000000,
        "description": "An attacker exploited a signature verification vulnerability in the Wormhole bridge, stealing 120,000 wETH ($320 million)",
        "exploit_addresses": [
            "EWuhy7xoH9AXtMwUGpg3wKVgtXqQJ6uGADDQPaczCgVJ",
            "5cQ8Aw9yuQHkKChy28XkWWxHfwK3PBsBKHqBRhvYNdD3"
        ],
        "vulnerable_contracts": [
            "worm2ZoG2kUd4vFXhvjh93UUH596ayRfgQ2MgjNMTth"
        ],
        "attack_vector": "signature_verification_bypass",
        "references": [
            "https://blog.chainalysis.com/reports/wormhole-hack-february-2022/"
        ]
    },
    {
        "name": "Mango Markets Exploit",
        "date": "2022-10-11",
        "type": "price_manipulation",
        "loss_usd": 114000000,
        "description": "Avraham Eisenberg manipulated the MNGO token price to extract $114 million from Mango Markets",
        "exploit_addresses": [
            "DKZYkQzTq4UgQx8hK3YHyJjnhGckZbWUxWoFCzga7VdC",
            "4gttw8ZNaL1MhWuSxPbEfAXrKLPQUQpHYP92iwKpgAhE"
        ],
        "vulnerable_contracts": [
            "mv3ekLzLbnVPNxjSKvqBpU3ZeZXPQdEC3bp5MDEBG68"
        ],
        "attack_vector": "oracle_manipulation",
        "references": [
            "https://blog.chainalysis.com/reports/mango-markets-hack-october-2022/"
        ]
    },
    {
        "name": "Solend Protocol Exploit",
        "date": "2022-11-02",
        "type": "oracle_exploit",
        "loss_usd": 1260000,
        "description": "An attacker exploited a faulty price oracle to manipulate an illiquid token price and drain $1.26 million",
        "exploit_addresses": [
            "FkuEG9zVQggkDQUvkVWRHKjP2gRRX5BzQQJwmAUyuNJV"
        ],
        "vulnerable_contracts": [
            "So1endDq2YkqhipRh3WViPa8hdiSpxWy6z3Z6tMCpAo"
        ],
        "attack_vector": "oracle_manipulation",
        "references": [
            "https://solend.fi/incident-report-11-02-22"
        ]
    },
    {
        "name": "Cashio Infinite Mint Exploit",
        "date": "2022-03-23",
        "type": "infinite_mint",
        "loss_usd": 52000000,
        "description": "An attacker exploited a verification flaw in Cashio's audited contracts to mint unlimited CASH tokens and drain $52 million",
        "exploit_addresses": [
            "GKycmhvaN7SZWLFYpbwSRsyWDGqJFUcVCfkrJkn8fGr8"
        ],
        "vulnerable_contracts": [
            "CASHVDm2wsJXfhj6VWxb7GiMdoLc17Du7paH4bNr5woT"
        ],
        "attack_vector": "verification_bypass",
        "references": [
            "https://halborn.com/explained-the-cashio-hack-march-2022/"
        ]
    },
    {
        "name": "Crema Finance Exploit",
        "date": "2022-07-02",
        "type": "flash_loan_attack",
        "loss_usd": 9000000,
        "description": "An attacker used a flash loan to manipulate a liquidity pool and steal $9 million in crypto",
        "exploit_addresses": [
            "DdZR6zRFiUt4S5mg7AV1uKB2z1f1WzcNYCaTEEWPAuUS"
        ],
        "vulnerable_contracts": [
            "Cr2yCRvNGZT9UkcGDSFr2ErYY2uMMwA7JYz14XeizP1r"
        ],
        "attack_vector": "flash_loan",
        "references": [
            "https://crema.finance/post/6tvgotkzshbgcbwy2vk2jr"
        ]
    }
]

# Convert to DataFrame for easier analysis
incidents_df = pd.DataFrame(SECURITY_INCIDENTS)
incidents_df['date'] = pd.to_datetime(incidents_df['date'])

# Display incidents summary
print(f"Loaded {len(incidents_df)} security incidents with total losses of ${incidents_df['loss_usd'].sum()/1000000:.1f} million")
incidents_df[['name', 'date', 'type', 'loss_usd']]

## Analyze Incident Timeline

Let's visualize the timeline of security incidents and analyze patterns:

In [ ]:
# Plot the incident timeline
timeline_image = plot_incident_timeline(incidents_df)
print(f"Timeline saved to: {timeline_image}")

# Display the timeline in notebook
from IPython.display import Image, display
display(Image(timeline_image))

# Analyze temporal patterns
incidents_by_month = incidents_df.groupby(incidents_df['date'].dt.to_period('M')).agg({
    'loss_usd': 'sum',
    'name': 'count'
})

incidents_by_month.columns = ['Total Loss (USD)', 'Number of Incidents']
incidents_by_month['Average Loss (USD)'] = incidents_by_month['Total Loss (USD)'] / incidents_by_month['Number of Incidents']

print("\nIncidents by month:")
display(incidents_by_month)

## Exploiter Transactions Analysis

Let's analyze transactions from exploit addresses to understand the attack patterns:

In [ ]:
# Extract all exploit addresses from incidents
exploit_addresses = [addr for incident in SECURITY_INCIDENTS for addr in incident.get('exploit_addresses', [])]

print(f"Found {len(exploit_addresses)} unique exploit addresses across all incidents")

# Let's analyze the first exploit (Wormhole hack)
wormhole_incident = SECURITY_INCIDENTS[0]
wormhole_addresses = wormhole_incident['exploit_addresses']

# Fetch transaction data for the exploit addresses
# In a real scenario, we would fetch this from an API
# For this example, we'll simulate it

print(f"Analyzing exploiter addresses for {wormhole_incident['name']}:")
for address in wormhole_addresses:
    print(f"- {address}")

# Simulate transaction data (in a real scenario, we would use API collectors)
# This format should match what would be returned by API collectors
sample_transactions = pd.DataFrame({
    'transaction_id': ['tx1', 'tx2', 'tx3', 'tx4', 'tx5'],
    'block_time': pd.date_range(start='2022-02-02', periods=5, freq='10min'),
    'address': [wormhole_addresses[0], wormhole_addresses[0], wormhole_addresses[1], 
               wormhole_addresses[1], wormhole_incident['vulnerable_contracts'][0]],
    'signer': [wormhole_addresses[0], wormhole_addresses[0], wormhole_addresses[1], 
              wormhole_addresses[0], wormhole_addresses[1]],
    'instruction_type': ['createAccount', 'transfer', 'invoke', 'transfer', 'close'],
    'amount': [120000, 120000, 0, 120000, 0],
    'success': [True, True, True, True, True]
})

print("\nSample transactions for analysis:")
display(sample_transactions)

## Transaction Entropy Analysis

Entropy analysis helps identify unusual patterns in transaction behavior that may indicate an exploit:

In [ ]:
# Calculate transaction entropy
entropy_results = calculate_transaction_entropy(sample_transactions)

print("Transaction entropy analysis:")
display(entropy_results)

# Detect anomalies in entropy
anomalies = detect_entropy_anomalies(entropy_results)

print("\nDetected entropy anomalies:")
anomalous_entries = anomalies[anomalies['anomaly_flag'] == True]
display(anomalous_entries if not anomalous_entries.empty else "No anomalies detected")

## Detailed Incident Analysis

Let's perform a detailed analysis of the Wormhole exploit:

In [ ]:
# Perform detailed analysis on the incident
incident_analysis = analyze_security_incident(
    wormhole_incident,
    transactions=sample_transactions
)

print("Incident Analysis Results:")
print(f"Incident: {incident_analysis['incident']['name']}")
print(f"Date: {incident_analysis['incident']['date']}")
print(f"Loss: ${incident_analysis['incident']['loss_usd']/1000000:.1f} million")
print(f"Attack Vector: {incident_analysis['incident']['attack_vector']}")

print("\nTransaction Patterns:")
for pattern in incident_analysis['transaction_patterns']:
    print(f"- {pattern['type']}: {pattern['description']}")

print("\nRecommendations:")
for rec in incident_analysis['recommendations']:
    print(f"- {rec['priority'].upper()}: {rec['title']} - {rec['description']}")

## Vulnerability Pattern Analysis

Let's analyze the vulnerability patterns in affected contracts:

In [ ]:
# Identify vulnerability patterns
vulnerability_patterns = identify_vulnerability_patterns(
    transactions=sample_transactions,
    contracts=wormhole_incident['vulnerable_contracts']
)

print("Vulnerability Pattern Analysis:")
print(json.dumps(vulnerability_patterns, indent=2))

## Transaction Flow Visualization

Let's visualize the transaction flow during the exploit:

In [ ]:
# Create a transaction flow graph using our utility classes
flow_graph = TransactionFlowGraph()

# For demonstration, we'll create a sample graph structure
# In a real scenario, this would be populated from actual transaction data
sample_graph_data = {
    "nodes": [
        {"id": wormhole_addresses[0], "type": "address", "risk_score": 90, "label": "Exploiter 1"},
        {"id": wormhole_addresses[1], "type": "address", "risk_score": 85, "label": "Exploiter 2"},
        {"id": wormhole_incident['vulnerable_contracts'][0], "type": "contract", "risk_score": 40, "label": "Wormhole Bridge"},
        {"id": "ETH_BRIDGE_ENDPOINT", "type": "bridge", "risk_score": 30, "label": "ETH Bridge"},
        {"id": "MIXER_SERVICE", "type": "mixer", "risk_score": 95, "label": "Mixer Service"}
    ],
    "edges": [
        {"source": wormhole_incident['vulnerable_contracts'][0], "target": wormhole_addresses[0], "volume": 120000, "weight": 3},
        {"source": wormhole_addresses[0], "target": wormhole_addresses[1], "volume": 60000, "weight": 2},
        {"source": wormhole_addresses[1], "target": "ETH_BRIDGE_ENDPOINT", "volume": 40000, "weight": 1},
        {"source": wormhole_addresses[0], "target": "MIXER_SERVICE", "volume": 30000, "weight": 1}
    ]
}

# Visualize the transaction flow
viz_file = visualize_transaction_flow(
    sample_graph_data,
    source_address=wormhole_incident['vulnerable_contracts'][0],
    highlight_paths=[[wormhole_incident['vulnerable_contracts'][0], wormhole_addresses[0], "MIXER_SERVICE"]]
)

print(f"Transaction flow visualization saved to: {viz_file}")

# Display the visualization
from IPython.display import Image
display(Image(viz_file))

## Common Attack Vectors Analysis

Let's analyze the distribution of attack vectors across incidents:

In [ ]:
# Analyze attack vectors
attack_vectors = incidents_df['attack_vector'].value_counts()

# Plot attack vector distribution
plt.figure(figsize=(10, 6))
ax = attack_vectors.plot(kind='bar', color=sns.color_palette("viridis", len(attack_vectors)))
plt.title('Distribution of Attack Vectors in Solana Security Incidents')
plt.xlabel('Attack Vector')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(attack_vectors):
    ax.text(i, v + 0.1, str(v), ha='center')

plt.tight_layout()
plt.show()

# Calculate total loss by attack vector
loss_by_vector = incidents_df.groupby('attack_vector')['loss_usd'].sum().sort_values(ascending=False) / 1000000

# Plot loss by attack vector
plt.figure(figsize=(10, 6))
ax = loss_by_vector.plot(kind='bar', color=sns.color_palette("plasma", len(loss_by_vector)))
plt.title('Total Losses by Attack Vector (USD millions)')
plt.xlabel('Attack Vector')
plt.ylabel('Loss Amount (USD millions)')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(loss_by_vector):
    ax.text(i, v + 5, f"${v:.1f}M", ha='center')

plt.tight_layout()
plt.show()

## Security Recommendations

Based on our analysis, here are key security recommendations for Solana protocols:

In [ ]:
# Define security recommendations based on our analysis
security_recommendations = {
    "signature_verification": [
        "Implement multi-signature verification for critical operations",
        "Always verify signatures on-chain rather than relying on client-side verification",
        "Validate signature authority against the expected public key"
    ],
    "oracle_security": [
        "Use Time-Weighted Average Price (TWAP) for price feeds",
        "Implement circuit breakers for unusual price movements",
        "Use multiple independent oracle sources and take median values"
    ],
    "access_control": [
        "Implement role-based access control for administrative functions",
        "Use Program Derived Addresses (PDAs) with proper seeds",
        "Add time-locks for sensitive operations"
    ],
    "program_logic": [
        "Follow checks-effects-interactions pattern",
        "Implement proper bounds checking for all inputs",
        "Avoid complex state transitions that can lead to inconsistent states"
    ],
    "monitoring": [
        "Implement real-time monitoring for unusual transaction patterns",
        "Set up alerts for high-value movements",
        "Monitor for entropy anomalies in transaction behavior"
    ]
}

# Display recommendations
for category, items in security_recommendations.items():
    print(f"\n{category.upper()}:")
    for i, item in enumerate(items, 1):
        print(f"{i}. {item}")

## Conclusion

This analysis has provided insights into major security incidents on Solana, their attack vectors, and patterns. Key findings include:

1. Signature verification bypasses and oracle manipulation are among the most financially damaging attack vectors
2. Many attacks follow similar transaction patterns that can be detected through entropy analysis
3. Transaction flow visualization helps identify suspicious fund movements

Future work should focus on:

- Building real-time monitoring systems for early detection
- Creating standardized security testing frameworks for Solana programs
- Developing more sophisticated transaction pattern recognition using machine learning